# Data EDA

* 데이터 라벨

|0|1|2|3|4|5|6|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|생활문화|스포츠|사회|세계|IT과학|경제|정치|

# Library import 단계

In [38]:
import transformers
import torch
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_huggingface.llms import HuggingFacePipeline
from tqdm import tqdm

tqdm.pandas()

# Noise vs Non - Noise 분류
- Ko-Llama3 사용

In [2]:
# train_data 불러오기
data = pd.read_csv('./data/train.csv')
data.head()

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6


- 초기 실행 시 10m 13.4s 소요(m2max, ram 32GB 기준). 
- 재실행시 2m 
- 모델 용량 4.89 + 4.92 + 5.00 + 1.55GB

In [3]:
model_id = "saltlux/Ko-Llama3-Luxia-8B"

pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="auto",
    max_length=100,
    max_new_tokens=50,
)
pipeline("<|begin_of_text|>안녕하세요. 솔트룩스 AI Labs 입니다.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "<|begin_of_text|>안녕하세요. 솔트룩스 AI Labs 입니다. 이번 포스팅에서는 솔트룩스 AI Labs에서 진행한 '2020년 1분기 AI 기술 트렌드'를 소개해 드리려고 합니다. 2020년 1분기 AI 기술 트렌드에서는 2020"}]

In [46]:
llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
class Result(BaseModel):
    cleaned_text: str = Field(description="노이즈가 제거된 뉴스 기사 제목")
    label: str = Field(description="뉴스 기사의 주제: (생활문화, 스포츠, 사회, 세계, IT과학, 경제, 정치) 중 하나")
    status: str = Field(description="데이터의 상태로 텍스트에 노이즈가 있으면 noise, 잘못 분류된 경우 mis-label, 정상인 경우 ok")

In [43]:
prompt_template = """
다음은 한국어 뉴스 제목입니다:
"{text}"
이 제목이 노이즈가 포함되어 있다면, {target} 주제를 참고하여 노이즈를 제거하고 올바른 문장으로 수정하세요.

"""

prompt = PromptTemplate(
    input_variables=['text', 'target'],
    template=prompt_template,
    partial_variables={
        'format_instructions': format_instructions
    }
)

In [44]:
chain = prompt | llm | output_fixing_parser

In [36]:
labels = ["생활문화", "스포츠", "사회", "세계", "IT과학", "경제", "정치"]

def detect_and_clean_noise(df: pd.DataFrame):
    def process_text(row):
        label_name = labels[row['target']]  # 현재 타겟의 라벨 이름을 가져옴
        result = chain.invoke({'text': row['text'], 'target': row['target'], 'label_name': label_name})
        
        if result['status'] == "discard":
            return None  # 노이즈가 너무 심해 복원할 수 없는 경우
        return result

    # 데이터 프레임에 적용
    df['processed_data'] = df.progress_apply(process_text, axis=1)
    df = df.dropna(subset=['processed_data'])  # 복원 불가한 데이터 삭제
    df['cleaned_text'] = df['processed_data'].apply(lambda x: x['cleaned_text'])
    df['label'] = df['processed_data'].apply(lambda x: x['label'])
    df['status'] = df['processed_data'].apply(lambda x: x['status'])
    df.drop(columns=['processed_data'], inplace=True)
    
    return df


noise_detected_data = detect_and_clean_noise(data)
noise_detected_data

  0%|          | 0/2800 [00:00<?, ?it/s]Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


파싱 오류 발생: Ambiguous response. Both True and False in received: 
다음은 한국어 뉴스 제목입니다:
"정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보"
이 제목이 노이즈가 포함되어 있다면 True, 그렇지 않다면 False로 답변하세요.
오직 하나의 True 혹은 False로만 답변하세요. 다른 텍스트는 출력하지 마세요.
예제:
"True"
"False"
"True"
"False"
"True"
"False"
"True"
"False"
"True"
"False"
"True"
"False"
"True"
"False"
"True"
"False.. 1/3번째 재시도 중...


Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


파싱 오류 발생: Ambiguous response. Both True and False in received: 
다음은 한국어 뉴스 제목입니다:
"정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보"
이 제목이 노이즈가 포함되어 있다면 True, 그렇지 않다면 False로 답변하세요.
오직 하나의 True 혹은 False로만 답변하세요. 다른 텍스트는 출력하지 마세요.
예제:
"True"
"False"
"True"
"False"
"True"
"False"
"True"
"False"
"True"
"False"
"True"
"False"
"True"
"False"
"True"
"False.. 2/3번째 재시도 중...


Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

파싱 오류 발생: Ambiguous response. Both True and False in received: 
다음은 한국어 뉴스 제목입니다:
"美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다"
이 제목이 노이즈가 포함되어 있다면 True, 그렇지 않다면 False로 답변하세요.
오직 하나의 True 혹은 False로만 답변하세요. 다른 텍스트는 출력하지 마세요.
예시:
True
False
입력
입력
입력
입력
입력
입력
입력
입력
입력
입력
입력
입력
입력
입력
입.. 3/3번째 재시도 중...


  0%|          | 7/2800 [00:57<7:06:32,  9.16s/it]Both `max_new_tokens` (=50) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


최대 재시도 횟수 초과: '美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다' 처리 실패.


  0%|          | 7/2800 [00:58<6:29:46,  8.37s/it]


KeyboardInterrupt: 

In [ ]:
noise_data_count = len(data[data['is_noisy'] == True])
noise_data_count